In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from utils.util_funcs import build_columns_for_df, save_to_csv, read_from_csv

In [4]:
tickers = ['PG', "NIO", "BEI.DE"]

sec_data = pd.DataFrame()
for t in tickers:
    my_data = yf.download(t, start='2007-01-01')
    columns = build_columns_for_df(my_data)
    save_to_csv(my_data, t)
    sec_data[t] = read_from_csv(t, columns)['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
sec_data.tail()

,PG,NIO,BEI.DE
Date,,,
2025-03-24,165.649994,4.38,127.199997
2025-03-25,162.850006,4.39,118.699997
2025-03-26,166.580002,4.22,118.050003
2025-03-27,168.710007,3.98,119.849998
2025-03-28,168.029999,3.75,120.349998


In [6]:
sec_returns = np.log(sec_data / sec_data.shift(1))

In [7]:
sec_returns

,PG,NIO,BEI.DE
Date,,,
2007-01-03,NaN,NaN,NaN
2007-01-04,-0.007621,NaN,0.006544
2007-01-05,-0.008624,NaN,-0.020771
2007-01-08,0.002202,NaN,0.000202
2007-01-09,-0.002517,NaN,-0.022858
...,...,...,...
2025-03-24,-0.006259,-0.027029,0.000000
2025-03-25,-0.017048,0.002280,-0.069161
2025-03-26,0.022646,-0.039494,-0.005491


## PG

In [8]:
sec_returns["PG"].mean()

np.float64(0.00032182754688337773)

In [9]:
sec_returns["PG"].mean() * 250

np.float64(0.08045688672084443)

In [11]:
sec_returns["PG"].std()

np.float64(0.011635825144691496)

In [12]:
sec_returns["PG"].std() * 250 ** 0.5

np.float64(0.18397854956341708)

## Biersdorf

In [13]:
sec_returns["BEI.DE"].mean()

np.float64(0.00022322542702432846)

In [14]:
sec_returns["BEI.DE"].mean() * 250

np.float64(0.055806356756082115)

In [15]:
sec_returns["BEI.DE"].std()

np.float64(0.01342850842590003)

In [16]:
sec_returns["BEI.DE"].std() * 250 ** 0.5

np.float64(0.21232336102303256)

## NIO

In [17]:
sec_returns["NIO"].mean()

np.float64(-0.00034386483856475045)

In [18]:
sec_returns["NIO"].mean() * 250

np.float64(-0.08596620964118762)

In [19]:
sec_returns["NIO"].std()

np.float64(0.05445447256432502)

In [20]:
sec_returns["NIO"].std() * 250 ** 0.5

np.float64(0.8610008104320847)

In [21]:
print(sec_returns["NIO"].mean() * 250)
print(sec_returns["BEI.DE"].mean() * 250)
print(sec_returns["NIO"].mean() * 250)


-0.08596620964118762
0.055806356756082115
-0.08596620964118762


In [23]:
sec_returns["PG", "BEI.DE", "NIO"].mean() * 250

KeyError: ('PG', 'BEI.DE', 'NIO')

In [24]:
sec_returns[["PG", "BEI.DE", "NIO"]].mean() * 250

PG        0.080457
BEI.DE    0.055806
NIO      -0.085966
dtype: float64

## This is the risk nd high volatility

In [25]:
sec_returns[["PG", "BEI.DE", "NIO"]].std() * 250 ** 0.5

PG        0.183979
BEI.DE    0.212323
NIO       0.861001
dtype: float64

## Covariance and Correlation


Calculate variance

In [26]:
pg_var = sec_returns["PG"].var()
pg_var

np.float64(0.00013539242679783486)

In [27]:
bei_de_var = sec_returns["BEI.DE"].var()
bei_de_var

np.float64(0.0001803248385444681)

In [28]:
nio_var = sec_returns["NIO"].var()
nio_var

np.float64(0.002965289582258826)

In [29]:
pg_var_a = sec_returns["PG"].var() * 250
pg_var_a

np.float64(0.033848106699458716)

In [30]:
bei_de_var_a = sec_returns["BEI.DE"].var() * 250
bei_de_var_a

np.float64(0.045081209636117024)

In [31]:
nio_var_a = sec_returns["NIO"].var() * 250
nio_var_a

np.float64(0.7413223955647065)

Calculate covariance

In [32]:
cov_matrix = sec_returns.cov()
cov_matrix

,PG,NIO,BEI.DE
PG,0.000135,0.000021,0.000042
NIO,0.000021,0.002965,0.000042
BEI.DE,0.000042,0.000042,0.000180


In [33]:
cov_matrix_a = sec_returns.cov() * 250
cov_matrix_a

,PG,NIO,BEI.DE
PG,0.033848,0.005174,0.010396
NIO,0.005174,0.741322,0.010522
BEI.DE,0.010396,0.010522,0.045081


Correlation

Note: the following cell is not the correlation between the price of the three equities!

In [34]:
corr_matrix = sec_returns.corr()
corr_matrix

,PG,NIO,BEI.DE
PG,1.000000,0.029403,0.265523
NIO,0.029403,1.000000,0.060234
BEI.DE,0.265523,0.060234,1.000000


## Calculating Portfolio Risk

Equal weighting schema

In [35]:
# here we decide how many percent we are investing in our tickers
weights = np.array([0.33,0.33,0.33])

Portfolio Variance

In [36]:
pfolio_var = np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))
pfolio_var

np.float64(0.09500834077427761)

Portfolio Volatility

In [37]:
pfolio_vol = (np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))) * 0.5
pfolio_vol

np.float64(0.047504170387138805)

In [38]:
print(str(round(pfolio_vol, 5) * 100) + ' %')

4.75 %


## Calculating Diversifiable and Non-diversifiable Risk of a Portfolio

In [39]:
# weights = np.array([0.33,0.33,0.33])
print(weights[0])
print(weights[1])
print(weights[2])

0.33
0.33
0.33


Diversifiable Risk:

In [40]:
# pg_var_a = sec_returns['PG'].var() * 250
pg_var_a

np.float64(0.033848106699458716)

In [41]:
# bei_de_var_a = sec_returns['BEI.DE'].var() * 250
bei_de_var_a

np.float64(0.045081209636117024)

In [42]:
# nio_var_a = sec_returns['NIO'].var() * 250
nio_var_a

np.float64(0.7413223955647065)

In [43]:
dr = pfolio_var - (weights[0] ** 2 * pg_var_a) - (weights[1] ** 2 * bei_de_var_a) - (weights[2] ** 2 * nio_var_a)
dr

np.float64(0.0056829293483368765)

In [44]:
print(str(round(dr*100, 3)) + ' %')

0.568 %


## Non-Diversifiable Risk

In [46]:
n_dr_1 = pfolio_var - dr
n_dr_1

np.float64(0.08932541142594073)

In [47]:
n_dr_2 = (weights[0] ** 2 * pg_var_a) + (weights[1] ** 2 * bei_de_var_a) + (weights[2] ** 2 * nio_var_a)
n_dr_2

np.float64(0.08932541142594073)

In [48]:
n_dr_1 == n_dr_2

np.True_